In [ ]:
import cv2 as cv

def IoU(box1, box2):
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    return iou

def TemplateMatching(img):
    methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR', 'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']

    template = cv.imread('template.png', cv.IMREAD_GRAYSCALE)
    w, h = template.shape[::-1]
    
    vote = 0
    for meth in methods:
        method = eval(meth)

        res = cv.matchTemplate(img,template,method)
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)

        if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
            top_left = min_loc
        else:
            top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        box_xy = (top_left[0], top_left[1], bottom_right[0], bottom_right[1])
        # print(box_xy)
        iou = IoU((30, 525, 130, 555), box_xy)
        # print(iou)
        # hyperparameter: iou threshold
        if iou >= 0.95:
            vote += 1
    # emsemble
    # hyperparameter: vote threshold
    if vote >= 5:
        return 1
    else:
        return 0